In [1]:
!pip install tensorflow tensorflow_gpu pandas matplotlib sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
np.expand_dims

<function numpy.expand_dims(a, axis)>

In [4]:
df=pd.read_csv('/content/train (2).csv')

In [5]:
from tensorflow.keras.layers import TextVectorization
x=df['comment_text']
y=df[df.columns[2:]].values
MAX_FEATURES=10000 #no.of words in the vocab
vectorizer=TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [6]:
vectorizer.adapt(x.values)

In [7]:
vectorized_text=vectorizer(x.values)

In [9]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset =dataset.prefetch(8)

In [10]:
batch_x, batch_y=dataset.as_numpy_iterator().next()
train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [11]:
train.as_numpy_iterator().next()

(array([[ 296, 3364,   16, ...,    0,    0,    0],
        [ 124,    7,    5, ...,    0,    0,    0],
        [   1, 4325, 4767, ...,    0,    0,    0],
        ...,
        [4018,    1, 1038, ...,    0,    0,    0],
        [ 920,   49,  234, ...,    0,    0,    0],
        [   1,  171, 2788, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [13]:
model=Sequential()
#creating the embedding layer
model.add(Embedding(MAX_FEATURES+2, 32))
#Bidirectional LSTM layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
#Feature extractor fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
#final layer
model.add(Dense(6, activation='sigmoid'))

In [14]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320064    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [16]:
history=model.fit(train, epochs=1, validation_data=val) #for better results set epochs=5

6981/6981 [==============================] - 825s 117ms/step - loss: 0.0627 - val_loss: 0.0593


In [17]:
#predicition time
input_text=vectorizer('you freaking suck')
print(input_text)
batch=test.as_numpy_iterator().next()
test.as_numpy_iterator().next()
model.predict(np.expand_dims(input_text, 0 ))
res=model.predict(np.expand_dims(input_text, 0 ))

tf.Tensor([   7 7158  397 ...    0    0    0], shape=(1800,), dtype=int64)
1/1 [==============================] - 0s 55ms/step


In [18]:
#evaluation time
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [19]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [20]:
for batch in test.as_numpy_iterator():
  #unpack the batch
  x_true, y_true=batch
  #make a prediction
  yhat=model.predict(x_true)
  #flatten the prediction
  y_true= y_true.flatten()
  yhat=yhat.flatten()

  pre.update_state(y_true, yhat)
  re.update_state(y_true, yhat)
  acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 54ms/step


In [21]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8600543737411499, Recall:0.5365922451019287, Accuracy:0.4393179416656494


In [22]:
!pip install gradio jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [23]:
import gradio as gr

In [24]:
model.save('toxicity.h5')

In [25]:
model=tf.keras.models.load_model('toxicity.h5')

In [26]:
def score_comment(comment):
  vectorized_comment=vectorizer([comment])
  results=model.predict(vectorized_comment)
  text=''
  for idx, col in enumerate(df.columns[2:-1]):
    text+='{}: {}\n'.format(col, results[0][idx]>0.5)
  return text

In [28]:
interface=gr.Interface(fn=score_comment,
                       inputs=gr.inputs.Textbox(lines=2, placeholder='comment to score'),
                       outputs='text')

<ipython-input-28-8fb7737f72ed>:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=2, placeholder='comment to score'),
<ipython-input-28-8fb7737f72ed>:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder='comment to score'),
<ipython-input-28-8fb7737f72ed>:2: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder='comment to score'),


In [29]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6da4625fea12c9ac86.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
